# Assignment 6
## Language model
The assignment and data are available here: https://snlp2018.github.io/assignments.html

The `train` folder in the archive contains several text file produced by students of the course as self-introduction; the `test` folder contains a few text files of the same kind.

### Excercise 1
Tokenization. Using `nltk`, we write a function which takes a variable number of text files as input and returns a tokenize corpus (list of lists of tokens) and the corpus' vocabulary.

In [1]:
from nltk import wordpunct_tokenize
from glob import glob
import numpy as np

In [2]:
input_path = "data/train/" # this is the argument of the function

In [3]:
def tokenize(path_to_files):
    input_files = glob(input_path + "*.txt") # get all the txt files in the folder

    texts = [] # initialize empty list where we collect data
    for input_file in input_files:
        with open(input_file, "r", encoding = "utf-8") as in_file: # read the file
            text = in_file.read().split(".") # split sentences
            for line in text:
                tknzd_line = wordpunct_tokenize(line.lower()) # lowercase and tokenize each sentence
                if tknzd_line != []:
                    tknzd_line.insert(0, "<s>") # add beginning-of-sentence,
                    tknzd_line.insert(0, "<s>") # (twice because we'll be dealing with 3-grams)
                    tknzd_line.append("</s>") # and end-of-sentence tags
                    texts.append(tknzd_line)
    
    tokens = [token for sentence in texts for token in sentence]
    vocab = sorted(set(tokens)) # list of unique tokens

    return(texts, tokens, vocab)

In [58]:
corpus, all_tokens, vocab = tokenize(input_path)

For example:

In [59]:
print(corpus[0:1])

[['<s>', '<s>', 'i', 'always', 'wanted', 'to', 'work', 'with', 'words', 'or', 'language', ',', 'and', 'it', 'didn', '’', 't', 'matter', 'for', 'me', 'would', 'it', 'be', 'something', 'related', 'to', 'neurolinguistics', ',', 'just', 'a', 'simple', 'translation', 'work', 'or', 'anything', 'more', 'computational', '</s>']]


In [60]:
print(vocab[0:100])

['!', '!“', '"', '#', "'", '(', ')', '),', ')?', ',', '-', '/', '/?', '1m', '1st', '2000', '2004', '2011', '2016', '20s', '3', ':', '://', ';', '</s>', '<s>', '=', '>', '?', '?",', '@', 'a', 'ability', 'able', 'about', 'above', 'abovementioned', 'access', 'achieved', 'achievement', 'acquainted', 'acquire', 'acquiring', 'acquisition', 'across', 'activity', 'actually', 'addictive', 'addition', 'addresses', 'advantaged', 'after', 'age', 'agricultural', 'ai', 'aid', 'aided', 'alexa', 'algorithms', 'aligned', 'all', 'allow', 'allowing', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'alter', 'although', 'always', 'am', 'amazigh', 'amazing', 'amazon', 'america', 'among', 'amount', 'an', 'anagrams', 'analyse', 'analysing', 'analysis', 'analytics', 'analyze', 'analyzing', 'and', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'apart', 'api', 'appealing', 'application', 'applications', 'apply']


In [61]:
len(all_tokens)

5841

Shall we remove punctuation as pre-processing? We'll see...

In [62]:
len(vocab)

1192

### Excercise 2
N-gram language model.

First, write a function to extract n-grams from a tokenized sentence:

In [63]:
def ngram_sentence(n, sentence): # very basic, accepts only n=1, n=2, n=3
    if n == 1:
        out_sentence = [(token) for token in sentence]
    elif n == 2:
        out_sentence = [(sentence[i], sentence[i+1]) for i in range(0, len(sentence) - (n-1))]
    else:
        out_sentence = [(sentence[i], sentence[i+1], sentence[i+2]) for i in range(0, len(sentence) - (n-1))]
    return(out_sentence)

For example:

In [64]:
print(ngram_sentence(1, sentence = corpus[1]))

['<s>', '<s>', 'in', 'the', 'end', 'i', 'got', 'a', 'bit', 'more', 'interested', 'in', 'computer', 'linguistics', ',', 'because', 'in', 'school', 'i', 'always', 'was', 'good', 'with', 'programming', 'and', 'it', 'felt', 'natural', 'to', 'continue', 'improving', 'myself', 'in', 'that', 'field', '</s>']


In [65]:
print(ngram_sentence(3, sentence = corpus[1]))

[('<s>', '<s>', 'in'), ('<s>', 'in', 'the'), ('in', 'the', 'end'), ('the', 'end', 'i'), ('end', 'i', 'got'), ('i', 'got', 'a'), ('got', 'a', 'bit'), ('a', 'bit', 'more'), ('bit', 'more', 'interested'), ('more', 'interested', 'in'), ('interested', 'in', 'computer'), ('in', 'computer', 'linguistics'), ('computer', 'linguistics', ','), ('linguistics', ',', 'because'), (',', 'because', 'in'), ('because', 'in', 'school'), ('in', 'school', 'i'), ('school', 'i', 'always'), ('i', 'always', 'was'), ('always', 'was', 'good'), ('was', 'good', 'with'), ('good', 'with', 'programming'), ('with', 'programming', 'and'), ('programming', 'and', 'it'), ('and', 'it', 'felt'), ('it', 'felt', 'natural'), ('felt', 'natural', 'to'), ('natural', 'to', 'continue'), ('to', 'continue', 'improving'), ('continue', 'improving', 'myself'), ('improving', 'myself', 'in'), ('myself', 'in', 'that'), ('in', 'that', 'field'), ('that', 'field', '</s>')]


Next, we write a function which counts the occurrences of each unique n-gram in a given sentence and updates a dictionary with the counts; the keys of the dictionary are the unique n-grams found in the sentence:

In [66]:
def ngram_update(unique_ngrams, sentence):
    for ngram in sentence:
        if ngram in unique_ngrams.keys():
            unique_ngrams[ngram] += 1
        else:
            unique_ngrams[ngram] = 1

For example:

In [67]:
example_ngrams = {} # initialize empty dict
ngram_update(example_ngrams, ngram_sentence(3, sentence = corpus[0]))

In [68]:
print(example_ngrams)

{('<s>', '<s>', 'i'): 1, ('<s>', 'i', 'always'): 1, ('i', 'always', 'wanted'): 1, ('always', 'wanted', 'to'): 1, ('wanted', 'to', 'work'): 1, ('to', 'work', 'with'): 1, ('work', 'with', 'words'): 1, ('with', 'words', 'or'): 1, ('words', 'or', 'language'): 1, ('or', 'language', ','): 1, ('language', ',', 'and'): 1, (',', 'and', 'it'): 1, ('and', 'it', 'didn'): 1, ('it', 'didn', '’'): 1, ('didn', '’', 't'): 1, ('’', 't', 'matter'): 1, ('t', 'matter', 'for'): 1, ('matter', 'for', 'me'): 1, ('for', 'me', 'would'): 1, ('me', 'would', 'it'): 1, ('would', 'it', 'be'): 1, ('it', 'be', 'something'): 1, ('be', 'something', 'related'): 1, ('something', 'related', 'to'): 1, ('related', 'to', 'neurolinguistics'): 1, ('to', 'neurolinguistics', ','): 1, ('neurolinguistics', ',', 'just'): 1, (',', 'just', 'a'): 1, ('just', 'a', 'simple'): 1, ('a', 'simple', 'translation'): 1, ('simple', 'translation', 'work'): 1, ('translation', 'work', 'or'): 1, ('work', 'or', 'anything'): 1, ('or', 'anything', 'more

We can apply this function to our corpus and get 1-gram, 2-gram and 3-gram counts:

In [69]:
counts_1grams = {}
for sentence in corpus:
    ngram_update(counts_1grams, ngram_sentence(1, sentence))

In [70]:
print(counts_1grams)

{'<s>': 478, 'i': 164, 'always': 11, 'wanted': 4, 'to': 169, 'work': 14, 'with': 39, 'words': 7, 'or': 16, 'language': 55, ',': 198, 'and': 169, 'it': 63, 'didn': 3, '’': 4, 't': 5, 'matter': 1, 'for': 40, 'me': 26, 'would': 15, 'be': 27, 'something': 7, 'related': 3, 'neurolinguistics': 1, 'just': 6, 'a': 127, 'simple': 3, 'translation': 7, 'anything': 1, 'more': 27, 'computational': 45, '</s>': 239, 'in': 153, 'the': 194, 'end': 2, 'got': 4, 'bit': 3, 'interested': 28, 'computer': 21, 'linguistics': 67, 'because': 12, 'school': 3, 'was': 24, 'good': 1, 'programming': 17, 'felt': 1, 'natural': 10, 'continue': 1, 'improving': 3, 'myself': 2, 'that': 49, 'field': 16, 'also': 32, 'allow': 3, 'solve': 1, 'bigger': 1, 'variety': 2, 'of': 129, 'tasks': 5, ':': 3, 's': 13, 'really': 8, 'impressive': 1, 'how': 37, 'studying': 9, 'one': 17, 'thing': 6, 'can': 32, 'help': 6, 'you': 1, '“': 3, 'get': 2, 'access': 1, '”': 2, 'so': 17, 'many': 8, 'completely': 1, 'different': 12, 'fields': 5, 'sta

In [71]:
counts_2grams = {}
for sentence in corpus:
    ngram_update(counts_2grams, ngram_sentence(2, sentence))

In [72]:
counts_3grams = {}
for sentence in corpus:
    ngram_update(counts_3grams, ngram_sentence(3, sentence))

Next, compute n-gram probabilities, i.e. conditional probabilities:

In [73]:
# this is the probability of each token in the corpus
prob_1grams = {}
prob_1grams = {ngram : (counts_1grams[ngram] / len(all_tokens)) for ngram in counts_1grams.keys()}

For example:

In [74]:
prob_1grams["<s>"]

0.08183530217428522

In [75]:
prob_1grams["i"]

0.0280773840095874

In [76]:
prob_1grams["you"]

0.0001712035610340695

In [77]:
# this is the probability of the second token, given the first, in each 2-gram
prob_2grams = {}
prob_2grams = {ngram : (counts_2grams[ngram] / counts_1grams[ngram[0]]) for ngram in counts_2grams.keys()}

In [78]:
prob_2grams[("<s>", "i")]

0.1192468619246862

In [79]:
prob_2grams[("i", "am")]

0.1402439024390244

In [80]:
prob_2grams[("i", "want")]

0.012195121951219513

In [81]:
# this is the probability of the third token, given the first two, in each 3-gram
prob_3grams = {}
prob_3grams = {ngram : (counts_3grams[ngram] / counts_2grams[(ngram[0], ngram[1])]) for ngram in counts_3grams.keys()}

In [82]:
prob_3grams[("<s>", "<s>", "i")]

0.2384937238493724

In [83]:
prob_3grams[("<s>", "i", "am")]

0.2807017543859649

In [84]:
prob_3grams[("and", "i", "am")]

0.16666666666666666

Next, we write a function which computes the MLE-probability of a tokenized sentence in a n-gram model:

In [85]:
def sentence_prob(n, sentence):
    p_sentence = 1
    ngrams = ngram_sentence(n, sentence)

    if n == 1:
        for ngram in ngrams:
            if ngram in prob_1grams.keys():
                p_ngram = prob_1grams[ngram]
            else:
                p_ngram = 0
            p_sentence = p_sentence * p_ngram

    elif n == 2:
        for ngram in ngrams:
            if ngram in prob_2grams.keys():
                p_ngram = prob_2grams[ngram]
            else:
                p_ngram = 0
            p_sentence = p_sentence * p_ngram

    else:
        for ngram in ngrams:
            if ngram in prob_3grams.keys():
                p_ngram = prob_3grams[ngram]
            else:
                p_ngram = 0
            p_sentence = p_sentence * p_ngram
    
    return(p_sentence)

For example:

In [86]:
sentence = corpus[0]
sentence_prob(1, sentence)

1.7192490764201554e-92

In [87]:
sentence_prob(2, sentence)

4.930060929596075e-38

In [88]:
sentence_prob(3, sentence)

1.4412823861917303e-09

Next, we define a similar function to computed smoothed probability, so that we can evaluate sentences containing unseen n-grams as well.

First, we need to compute the smoothed probabilities of ngrams:

In [89]:
# this is the laplace-smoothed (=add one) probability of each token in the corpus
sprob_1grams = {}
sprob_1grams = {ngram : ((counts_1grams[ngram]+1) / (len(all_tokens)+len(vocab))) for ngram in counts_1grams.keys()}

In [90]:
# in order to compute the good-turing-smoothed probability of n-grams, we need a function that, for each rate r
# (e.g. r=1 means once, r=2 twice etc) tells us how many n-grams occured with that rate r in the corpus

def rate_ngram(r, ngrams): # r is the rate, ngrams is a dictionary with counts of n-grams
    c = 0
    for key,value in ngrams.items():
        if value == r:
            c = c + 1
    return(c)

For example:

In [91]:
rate_ngram(2, counts_2grams)

315

Next, for each ngram in the dictionary, we compute its probability as `(r+1)*(N_(r+1))/(N*N_r)` where `r` is the rate of the n-gram, `N_(r)` is the result of `rate_ngram(r, dictionary)` defined above, and similarly `N_(r+1) = rate_ngram(r+1, dictionary)`; `N_ngrams` is the total number of n-grams: 

In [92]:
sprob_2grams = {}
N_2grams = len(counts_2grams.keys())
for ngram in counts_2grams.keys():
    r = counts_2grams[ngram]
    N_r = rate_ngram(r, counts_2grams)
    N_r1 = rate_ngram(r+1, counts_2grams)

    sprob_2grams[ngram] = (r+1)*(N_r1)/(N_2grams*N_r)

Same for corpus of 3-grams:

In [93]:
sprob_3grams = {}
N_3grams = len(counts_3grams.keys())
for ngram in counts_3grams.keys():
    r = counts_3grams[ngram]
    N_r = rate_ngram(r, counts_3grams)
    N_r1 = rate_ngram(r+1, counts_3grams)

    sprob_3grams[ngram] = (r+1)*(N_r1)/(N_3grams*N_r)

Finally, the smoothed probability function:

In [94]:
def sentence_sprob(n, sentence):
    p_sentence = 1
    ngrams = ngram_sentence(n, sentence)

    if n == 1:
        for ngram in ngrams:
            if ngram in sprob_1grams.keys():
                p_ngram = sprob_1grams[ngram]
            else:
                p_ngram = 1/len(all_tokens)
            p_sentence = p_sentence * p_ngram

    elif n == 2:
        for ngram in ngrams:
            if ngram in sprob_2grams.keys():
                p_ngram = sprob_2grams[ngram]
            else:
                p_ngram = rate_ngram(1, counts_2grams)/(N_2grams)
            p_sentence = p_sentence * p_ngram

    else:
        for ngram in ngrams:
            if ngram in sprob_3grams.keys():
                p_ngram = sprob_3grams[ngram]
            else:
                p_ngram = rate_ngram(1, counts_3grams)/(N_3grams)
            p_sentence = p_sentence * p_ngram
    
    return(p_sentence)

Let's look at some examples:

In [95]:
sentence_sprob(1, ["i", "love", "computational", "linguistics"])

8.438168826593244e-10

In [96]:
sentence_sprob(1, ["i", "luv", "computational", "linguistics"])

2.5400462830607033e-10

In [97]:
sentence_sprob(1, ["i", "love", "computational", "biology"])

2.4818143607627192e-11

In [98]:
sentence_sprob(1, ["i", "love", "computerazible", "languageness"])

3.9110042531093435e-13

With 2-grams:

In [99]:
sentence_sprob(2, ["i", "love", "computational", "linguistics"])

0.005228428326151054

In [100]:
sentence_sprob(2, ["i", "luv", "computational", "linguistics"])

0.005228428326151054

In [101]:
sentence_sprob(2, ["i", "love", "computational", "biology"])

0.6083649816642906

In [102]:
sentence_sprob(2, ["i", "love", "computerazible", "languageness"])

0.6083649816642906

With 3-grams:

In [103]:
sentence_sprob(3, ["i", "love", "computational", "linguistics"])

0.8983096627467306

In [104]:
sentence_sprob(3, ["i", "luv", "computational", "linguistics"])

0.8983096627467306

In [105]:
sentence_sprob(3, ["i", "love", "computational", "biology"])

0.8983096627467306

In [106]:
sentence_sprob(3, ["i", "love", "computerazible", "languageness"])

0.8983096627467306

Can this be right?

Next, perplexity. We compute the perplexity of testing corpus, using the smoothed probability above.

First, read test data:

In [107]:
input_path_test = "data/test/"

In [108]:
corpus_test, all_tokens_test, vocab_test = tokenize(input_path_test)

Next, for each model, compute smoothed probability of each sentence:

In [109]:
test_sprob_1grams = [sentence_sprob(1, sentence) for sentence in corpus_test]

In [110]:
test_sprob_2grams = [sentence_sprob(2, sentence) for sentence in corpus_test]

In [111]:
test_sprob_3grams = [sentence_sprob(3, sentence) for sentence in corpus_test]

Probability of full corpus is the product of the probabilities of the sentences; then perplexity is computed as `P_corpus^(-1/N)` where `N` is the number of tokens in the corpus:

In [115]:
P_corpus_1gram = np.prod(test_sprob_1grams)
test_ppl_1gram = P_corpus_1gram**(-(1/len(all_tokens_test)))
print(test_ppl_1gram)

inf


In [116]:
P_corpus_2gram = np.prod(test_sprob_2grams)
test_ppl_2gram = P_corpus_2gram**(-(1/len(all_tokens_test)))
print(test_ppl_2gram)

inf


In [117]:
P_corpus_3gram = np.prod(test_sprob_3grams)
test_ppl_3gram = P_corpus_3gram**(-(1/len(all_tokens_test)))
print(test_ppl_3gram)

inf


This doesn't look good :D

### Exercise 3
Neural language model. Let's try with this.